In [0]:
from pyspark.ml.classification import RandomForestClassificationModel
from pyspark.ml.feature import *

### Load model

In [0]:
model_path = 'abfss://model@netflixprojectdltp.dfs.core.windows.net/type_prediction'
model = RandomForestClassificationModel.load(model_path)

### Tạo dữ liệu để dự đoán

In [0]:
# Ví dụ lấy từ bộ Wednesday của Netflix và bộ phim điện ảnh Spider-Man No Way Home

df_test = spark.createDataFrame([
    (0, 2, 2022, "TV-14"),
    (148, 1, 2022, "PG-13"),
], ["duration_minutes", "duration_seasons", "release_year", "rating"])

# Tạo VectorAssembler để kết hợp các cột thành cột features
assembler = VectorAssembler(
    inputCols=["duration_minutes", "duration_seasons", "release_year", "ratingIndex"],  # Nếu cần, bạn có thể mã hóa cột 'rating'
    outputCol="features"
)

# Mã hóa cột 'rating' thành ratingIndex
ratingIndexer = StringIndexer(inputCol="rating", outputCol="ratingIndex")
df_test_with_rating = ratingIndexer.fit(df_test).transform(df_test)

# Tạo lại cột 'features' với cột đã mã hóa 'ratingIndex'
df_test_with_features = assembler.transform(df_test_with_rating)

### Dự đoán

0 là Movie và 1 là TV Show

In [0]:
predictions = model.transform(df_test_with_features)
predictions.select("features", "prediction", "probability").show()

+--------------------+----------+-----------+
|            features|prediction|probability|
+--------------------+----------+-----------+
|[0.0,2.0,2022.0,1.0]|       1.0|  [0.0,1.0]|
|[148.0,1.0,2022.0...|       0.0|  [1.0,0.0]|
+--------------------+----------+-----------+

